In [1]:
import pandas as pd

In [2]:
midnight_df = pd.read_csv('corpus/midnights_lyrics.csv', encoding = 'latin-1')

In [3]:
df = midnight_df[midnight_df['Album'] == 'Midnights']

In [4]:
df.head()

,index,Album,Song Name,Lyrics
134,142,Midnights,Lavender Haze,Meet me at midnight Staring at the ceiling wi...
135,143,Midnights,Maroon,When the morning came We were cleaning incense...
136,144,Midnights,Labyrinth,"""It only hurts this much right now"" Was what I..."
137,145,Midnights,Anti Hero,"I have this thing where I get older, but just ..."
138,146,Midnights,Vigilante Shit,Draw the cat eye sharp enough to kill a man Yo...


In [5]:
songs = df.groupby('Song Name').agg({'Lyrics': lambda x: " ".join(x)}).reset_index()

In [6]:
pd.options.display.max_colwidth = 5000

In [7]:
songs.head()

,Song Name,Lyrics
0,Anti Hero,"I have this thing where I get older, but just never wiser Midnights become my afternoons When my depression works the graveyard shift, all of the people I've ghosted stand there in the room I should not be left to my own devices They come with prices and vices I end up in crisis (Tale as old as time) I wake up screaming from dreaming One day, I'll watch as you're leaving 'Cause you got tired of my scheming (For the last time) It's me, hi I'm the problem, it's me At teatime, everybody agrees I'll stare directly at the sun, but never in the mirror It must be exhausting always rooting for the anti-hero Sometimes, I feel like everybody is a sexy baby And I'm a monster on the hill Too big to hang out, slowly lurching toward your favorite city Pierced through the heart, but never killed Did you hear my covert narcissism I disguise as altruism Like some kind of congressman? (A tale as old as time) I wake up screaming from dreaming One day, I'll watch as you're leaving And life will lose all its meaning (For the last time) It's me, hi I'm the problem, it's me (I'm the problem, it's me) At teatime, everybody agrees I'll stare directly at the sun, but never in the mirror It must be exhausting always rooting for the anti-hero You might also like I have this dream my daughter-in-law kills me for the money She thinks I left them in the will The family gathers 'round and reads it and then someone screams out ""She's laughing up at us from Hell"" It's me, hi I'm the problem, it's me It's me, hi I'm the problem, it's me It's me, hi Everybody agrees, everybody agrees It's me, hi (Hi) I'm the problem, it's me (I'm the problem, it's me) At teatime (Teatime), everybody agrees (Everybody agrees) I'll stare directly at the sun, but never in the mirror It must be exhausting always rooting for the anti-hero"
1,Bejeweled,"Baby love, I think I've been a little too kind Didn't notice you walkin' all over my peace of mind In the shoes I gave you as a present Puttin' someone first only works when you're in their top five And by the way, I'm goin' out tonight Best believe I'm still bejeweled When I walk in the room I can still make the whole place shimmer And when I meet the band They ask, ""Do you have a man?"" I could still say, ""I don't remember"" Familiarity breeds contempt Don't put mÐµ in the basement WhÐµn I want the penthouse of your heart Diamonds in my eyes I polish up real, I polish up real nice Nice Baby boy, I think I've been too good of a girl (Too good of a girl) Did all the extra credit, then got graded on a curve I think it's time to teach some lessons I made you my world (Huh), have you heard? (Huh) I can reclaim the land And I miss you (Miss you), but I miss sparklin' (Nice) Best believe I'm still bejeweled When I walk in the room I can still make the whole place shimmer And when I meet the band They ask, ""Do you have a man?"" I could still say, ""I don't remember"" Familiarity breeds contempt Don't put me in the basement When I want the penthouse of your heart Diamonds in my eyes I polish up real, I polish up real nice Nice You might also like Sapphire tears on my face Sadness became my whole sky But some guy said my aura's moonstone Just 'cause he was high And we're dancin' all night And you can try to change my mind But you might have to wait in line What's a girl gonna do? A diamond's gotta shine Best believe I'm still bejeweled When I walk in the room I can still make the whole place shimmer (Shimmer) And when I meet the band They ask, ""Do you have a man?"" I could still say, ""I don't remember"" Familiarity breeds contempt Don't put me in the basement When I want the penthouse of your heart Diamonds in my eyes I polish up real (Nice), I polish up real nice And we're dancin' all night And you can try to change my mind But you might have to wait in line What's a girl gonna do? What's a girl gonna do? I polish up nice Best believe I'm still bejeweled When I walk in the room I can still make the whole 

In [8]:
import sklearn

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/anita/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
stop_words = stopwords.words('english')

In [12]:
stop_words.extend(["still", "nice", "ask", "room", "put", "girl", "real", "best", "wish", "whole",
    "comin", "sweet", "like", "said", "end", "home", "somethin", "find", "ever", "everyone",
    "oh", "love", "right", "yeah", "uh", "thought", "goin", "around", "much", "stay",
    "get", "shit", "sad", "ends", "even", "want", "people", "someone",
    "everybody", "blood", "always", "wake", "kid", "never", "saw", "looked", "us",
    "wanted", "midnight", "told", "cause", "bride", "mine", "midnights"])

In [13]:
vectorizer = TfidfVectorizer(stop_words = stop_words, min_df = 0.1)

In [14]:
tfidf = vectorizer.fit_transform(songs['Lyrics'])

In [15]:
from sklearn.decomposition import NMF

In [16]:
nmf = NMF(n_components = 6)

In [17]:
topic_values = nmf.fit_transform(tfidf)

In [18]:
for topic_num, topic in enumerate(nmf.components_):
    message = "Topic #{}: ".format(topic_num + 1)
    message += " ".join([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11 :-1]])
    print(message)
    

Topic #1: night friends could make one fun first touch kiss remember
Topic #2: kind sometimes day time family thinks big heart things picture
Topic #3: feels dream stars fuckin ah tonight smile would life time
Topic #4: know cat loves back lap step keep town friends thieves
Topic #5: feel say new see need keep asking lost sky shoes
Topic #6: miss mind deep eye tired iâ picked last talking little


In [19]:
topic_labels = ["memories", "family", "aspirations", "companionship", "seeking understanding", "reflection"]

In [20]:
df_topics = pd.DataFrame(topic_values, columns = topic_labels)
df_topics

,memories,family,aspirations,companionship,seeking understanding,reflection
0,0.000000,0.627016,0.089695,0.000000,0.057370,0.017819
1,0.247289,0.000000,0.000000,0.000000,0.000589,0.220466
2,0.000000,0.000000,0.000000,0.870516,0.000000,0.000000
3,0.000000,0.000000,0.334292,0.281836,0.000000,0.259742
4,0.000295,0.000000,0.000000,0.000000,0.757179,0.000000
5,0.000000,0.096587,0.000000,0.001194,0.676257,0.000000
6,0.223603,0.000000,0.167861,0.000000,0.001840,0.000000
7,0.000000,0.740921,0.000000,0.000000,0.000000,0.000000
8,0.322351,0.000000,0.008663,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.828534,0.000000,0.000000,0.000000


In [21]:
songs = songs.join(df_topics)  #"memories", "family", "aspirations", "companionship", "seeking understanding", "reflection"]

In [22]:
songs.loc[songs['memories'] >= 0.1, 'memories'] = 1
songs.loc[songs['family'] >= 0.1, 'family'] = 1
songs.loc[songs['aspirations'] >= 0.1, 'aspirations'] = 1
songs.loc[songs['companionship'] >= 0.1, 'companionship'] = 1
songs.loc[songs['seeking understanding'] >= 0.1, 'seeking understanding'] = 1
songs.loc[songs['reflection'] >= 0.1, 'reflection'] = 1

In [23]:
songs.loc[songs['memories'] < 0.1, 'memories'] = 0
songs.loc[songs['family'] < 0.1, 'family'] = 0
songs.loc[songs['aspirations'] < 0.1, 'aspirations'] = 0
songs.loc[songs['companionship'] < 0.1, 'companionship'] = 0
songs.loc[songs['seeking understanding'] < 0.1, 'seeking understanding'] = 0
songs.loc[songs['reflection'] < 0.1, 'reflection'] = 0

In [24]:
songs_csv = songs.to_csv('topics/midnights_topics.csv')